# Running Processing Jobs

A processing job serves as a method to enhance the utilization of ordered geospatial data by employing advanced processing algorithms. This process transforms the data into an analytics-ready state or extracts information for subsequent insight derivation.

- <a href="#setup">Set up the notebook</a>
- <a href="#single-input">Run Single Input Process</a>
- <a href="#job-list">View Jobs</a>
- <a href="#two-input">Run Two Input Process</a>

## <a id="setup"></a> Set up the notebook

### 1. Install prerequisites

In [1]:
#!pip install up42-py
import up42, os
from up42 import processing_templates as templates
from up42 import processing as processes

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [3]:
# Define the directory path
up42_directory = os.path.expanduser("~/.up42")

# Create the directory if it doesn't exist
if not os.path.exists(up42_directory):
    os.makedirs(up42_directory)

# Specify the file path
credentials_file_path = os.path.join(up42_directory, "credentials.json")

# Check if the file already exists before creating it
if not os.path.exists(credentials_file_path):
    # Create an empty credentials.json file
    with open(credentials_file_path, "w") as credentials_file:
        print(f"The file {credentials_file_path} has been created.")
        pass
else:
    print(f"The file {credentials_file_path} already exists.")

The file /Users/pelle-svante.john/.up42/credentials.json already exists.


1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for username and password.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [4]:
up42.authenticate(cfg_file=credentials_file_path)

2024-06-24 14:45:23,643 - Authentication with UP42 successful!


## <a id="single-input"></a> Run Single Input Process

Execute Jobs with one STAC input such as Pansharpening

### 1. Initialize a specific STAC Item

A better scenario would be to have the asset_id OR STAC item_id already in mind 

In [6]:
asset_id = "358e6a5b-3bb4-43e2-8b7e-9b4098b89d5a" #to be cleared later
stac_item = up42.initialize_asset(asset_id).stac_items[0]

2024-06-24 14:45:50,894 - Initialized {'id': '358e6a5b-3bb4-43e2-8b7e-9b4098b89d5a', 'workspaceId': 'de807c20-faa1-4064-afe3-7aaf7c49830d', 'accountId': '3c86279b-610a-4dba-ad12-abe3bdc51428', 'createdAt': '2024-02-07T10:48:57.552117Z', 'updatedAt': '2024-02-07T10:49:52.095732Z', 'name': 'oneatlas_1314eb79-132a-4702-8ece-9e91e8080b1e.zip', 'size': 40595946, 'contentType': 'application/zip', 'geospatialMetadataExtractionStatus': 'SUCCESSFUL', 'productId': '4f1b2f62-98df-4c74-81f4-5dce45deee99', 'producerName': 'airbus', 'collectionName': 'phr', 'source': 'ARCHIVE', 'tags': []}


### 2. Prepare the parameters

In [7]:
title = "Example_Pansharpen_SDK"
expected_cost = 100 #MODIFY

### 3. Create a Job Template

In [8]:
template = templates.Pansharpening(
    title=title,
    item=stac_item
)
print(template)

Pansharpening(title='Example_Pansharpen_SDK', item=<Item id=2c13c8df-455a-4d55-810d-c5dc5353408f>, grey_weights=None, workspace_id='de807c20-faa1-4064-afe3-7aaf7c49830d')


### 4. Validate the Job

In [7]:
if not template.is_valid:
    print(template.errors)

### 5. Get the Job Pricing

In [8]:
print(template.cost)
assert template.cost.credits <= expected_cost

Cost(strategy='area', credits=21, size=5.039398810389698, unit='SQ_KM')


### 6. Execute & Track the Job

In [9]:
job = template.execute()
job.track(wait=30, retries=6)

print(job.status)
print(job.collection_url)

JobStatus.CAPTURED
https://api.up42.com/v2/assets/stac/collections/53a306fb-0387-4bb1-9cc6-1f7c8174bce0


## <a id="job-list"></a> View Jobs

Retrieve all jobs or a specific job available to user, with additional filtering capabilities

### 1. Get All Jobs

Set up parameters for filtering jobs

In [10]:
workspace_id = "de807c20-faa1-4064-afe3-7aaf7c49830d" #FILL IN
process_filter = [
    templates.Pansharpening.process_id,
    templates.DetectionChangeSpacept.process_id,
]
status_filter = [
    processes.JobStatus.CAPTURED, 
    processes.JobStatus.RELEASED
]
sort_filter = processes.JobSorting.credits.asc

Call the get jobs method, returning a generator object

In [21]:
jobs = processes.Job.all(
#    process_id=process_filter
#    workspace_id=workspace_id,
#    sort_by=status_filter,
#    min_duration=min_duration,
#    max_duration=max_duration,
)
for i in jobs:  
    print(i)

Job(process_id='pansharpening', id='e71a8f11-a207-4828-8547-8761e17c9347', account_id='3c86279b-610a-4dba-ad12-abe3bdc51428', workspace_id='de807c20-faa1-4064-afe3-7aaf7c49830d', definition={'inputs': {'item': 'https://api.up42.com/v2/assets/stac/collections/16d739e2-accc-461a-9088-ac7ff06a242f/items/2c13c8df-455a-4d55-810d-c5dc5353408f', 'title': 'Example_Pansharpen_SDK'}}, status=<JobStatus.CAPTURED: 'captured'>, created=datetime.datetime(2024, 6, 24, 8, 19, 40, 582000), updated=datetime.datetime(2024, 6, 24, 8, 20, 45, 665000), collection_url='https://api.up42.com/v2/assets/stac/collections/53a306fb-0387-4bb1-9cc6-1f7c8174bce0', errors=None, credits=21, started=datetime.datetime(2024, 6, 24, 8, 19, 52, 307000), finished=datetime.datetime(2024, 6, 24, 8, 20, 45, 460000))
Job(process_id='detection-buildings-spacept', id='943fb131-5ef0-48ed-a065-1c5451e8a053', account_id='3c86279b-610a-4dba-ad12-abe3bdc51428', workspace_id='de807c20-faa1-4064-afe3-7aaf7c49830d', definition={'inputs': {

TypeError: up42.processing.ValidationError() argument after ** must be a mapping, not str

### 2. Get a Specific Job

In [14]:
queried_job = processes.Job.get(job.id)
print(queried_job)

Job(process_id='pansharpening', id='e71a8f11-a207-4828-8547-8761e17c9347', account_id='3c86279b-610a-4dba-ad12-abe3bdc51428', workspace_id='de807c20-faa1-4064-afe3-7aaf7c49830d', definition={'inputs': {'item': 'https://api.up42.com/v2/assets/stac/collections/16d739e2-accc-461a-9088-ac7ff06a242f/items/2c13c8df-455a-4d55-810d-c5dc5353408f', 'title': 'Example_Pansharpen_SDK'}}, status=<JobStatus.CAPTURED: 'captured'>, created=datetime.datetime(2024, 6, 24, 8, 19, 40, 582000), updated=datetime.datetime(2024, 6, 24, 8, 20, 45, 665000), collection_url='https://api.up42.com/v2/assets/stac/collections/53a306fb-0387-4bb1-9cc6-1f7c8174bce0', errors=None, credits=21, started=datetime.datetime(2024, 6, 24, 8, 19, 52, 307000), finished=datetime.datetime(2024, 6, 24, 8, 20, 45, 460000))


## <a id="two-input"></a> Run Two Input Process

Execute Jobs with two STAC input such as Change Detection

In [ ]:
stac_item_id1= "" #FILL IN
stac_item_id2= "" #FILL IN

stac_item1 = pystac.get_item(stac_item_id1).to_dict()
stac_item2 = pystac.get_item(stac_item_id2).to_dict()

template = templates.HypervergePleiadesChangeDetection(
    title="SDK Detection Change Example", 
    items=[stac_item1, stac_item2]
)

And you would validate and execute the template in a similar manner to the first example